In [1]:
import numpy as np
import pandas as pd
import matplotlib
from tqdm import tqdm 
from sklearn import metrics 
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV


In [44]:
set_dir = './dream3/5/'

In [45]:
op_df = pd.read_csv(set_dir + 'Matrix_TF_gene_best_model.tsv', sep='\t', index_col=0)
inf_df = pd.read_csv(set_dir + 'inf_matrix.tsv', sep='\t', index_col=0)
boost_df = pd.read_csv(set_dir + 'GRNBOOST2/rankedEdges.csv', sep='\t')
genie3_df = pd.read_csv(set_dir + 'GENIE3/rankedEdges.csv', sep='\t')
grisli_df = pd.read_csv(set_dir + 'GRISLI/rankedEdges.csv', sep='\t')
grnvbem_df = pd.read_csv(set_dir + 'GRNVBEM/rankedEdges.csv', sep='\t')
leap_df = pd.read_csv(set_dir + 'LEAP/rankedEdges.csv', sep='\t')
pidc_df = pd.read_csv(set_dir + 'PIDC/rankedEdges.csv', sep='\t')
ppcor_df = pd.read_csv(set_dir + 'PPCOR/rankedEdges.csv', sep='\t')
scode_df = pd.read_csv(set_dir + 'SCODE/rankedEdges.csv', sep='\t')
scribe_df = pd.read_csv(set_dir + 'SCRIBE/rankedEdges.csv', sep='\t')
sincerities_df = pd.read_csv(set_dir + 'SINCERITIES/rankedEdges.csv', sep='\t')
# singe_df = pd.read_csv(set_dir + 'SINGE/rankedEdges.csv', sep='\t')

FileNotFoundError: [Errno 2] No such file or directory: './dream3/5/SINCERITIES/rankedEdges.csv'

In [ ]:
target_df = pd.read_csv(set_dir + 'target_edges.csv')
target_tf_set = set(target_df['regulator'])
target_gene_set = set(target_df['target'])
# common_tf = target_tf_set.intersection(set(op_df.columns))
# common_gene = target_gene_set.intersection(set(op_df.index))
common_tf = op_df.columns
common_gene = op_df.index
target_edge_set = set()
for index, row in target_df.iterrows():
    if row['regulator'] in common_tf and row['target'] in common_gene:
        target_edge_set.add(row['regulator']+'_'+row['target'])

In [ ]:
if 'Time' in common_tf:
    common_tf = common_tf.drop('Time')

In [ ]:
# ensemble_df = pd.DataFrame(columns=['edge_name', 'op', 'inf', 'grnboost', 'genie3', 'grisli', 'grnvbem', 'leap',  'pidc', 'ppcor', 'scode', 'scribe', 'sincerities', 'singe', 'ss_cor', 'ss_ranked', 'edge_exist'])
ensemble_df = pd.DataFrame(columns=['edge_name', 'op', 'inf', 'grnboost', 'genie3', 'grisli', 'grnvbem', 'leap',  'pidc', 'ppcor', 'scode', 'scribe', 'sincerities', 'ss_cor', 'ss_ranked', 'edge_exist'])
# ensemble_df = pd.DataFrame(columns=['edge_name', 'op', 'inf', 'grnboost', 'genie3', 'grisli', 'grnvbem', 'leap',  'pidc', 'ppcor', 'scode', 'scribe', 'ss_cor', 'ss_ranked', 'edge_exist'])

In [ ]:
boost_df = boost_df[boost_df['Gene1'].isin(common_tf)]
boost_df = boost_df[boost_df['Gene2'].isin(common_gene)]
genie3_df = genie3_df[genie3_df['Gene1'].isin(common_tf)]
genie3_df = genie3_df[genie3_df['Gene2'].isin(common_gene)]
grisli_df = grisli_df[grisli_df['Gene1'].isin(common_tf)]
grisli_df = grisli_df[grisli_df['Gene2'].isin(common_gene)]
grnvbem_df = grnvbem_df[grnvbem_df['Gene1'].isin(common_tf)]
grnvbem_df = grnvbem_df[grnvbem_df['Gene2'].isin(common_gene)]
leap_df = leap_df[leap_df['Gene1'].isin(common_tf)]
leap_df = leap_df[leap_df['Gene2'].isin(common_gene)]
pidc_df = pidc_df[pidc_df['Gene1'].isin(common_tf)]
pidc_df = pidc_df[pidc_df['Gene2'].isin(common_gene)]
ppcor_df = ppcor_df[ppcor_df['Gene1'].isin(common_tf)]
ppcor_df = ppcor_df[ppcor_df['Gene2'].isin(common_gene)]
scode_df = scode_df[scode_df['Gene1'].isin(common_tf)]
scode_df = scode_df[scode_df['Gene2'].isin(common_gene)]
scribe_df = scribe_df[scribe_df['Gene1'].isin(common_tf)]
scribe_df = scribe_df[scribe_df['Gene2'].isin(common_gene)]
sincerities_df = sincerities_df[sincerities_df['Gene1'].isin(common_tf)]
sincerities_df = sincerities_df[sincerities_df['Gene2'].isin(common_gene)]
# singe_df = singe_df[singe_df['Gene1'].isin(common_tf)]
# singe_df = singe_df[singe_df['Gene2'].isin(common_gene)]

In [ ]:
boost_matrix = pd.DataFrame(0.0, index=common_gene, columns=common_tf)
for index, row in boost_df.iterrows():
    boost_matrix.at[row['Gene2'], row['Gene1']] = np.abs(row['EdgeWeight'])
genie3_matrix = pd.DataFrame(0.0, index=common_gene, columns=common_tf)
for index, row in genie3_df.iterrows():
    genie3_matrix.at[row['Gene2'], row['Gene1']] = np.abs(row['EdgeWeight'])
grisli_matrix = pd.DataFrame(0.0, index=common_gene, columns=common_tf)
for index, row in grisli_df.iterrows():
    grisli_matrix.at[row['Gene2'], row['Gene1']] = np.abs(row['EdgeWeight'])
grnvbem_matrix = pd.DataFrame(0.0, index=common_gene, columns=common_tf)
for index, row in grnvbem_df.iterrows():
    grnvbem_matrix.at[row['Gene2'], row['Gene1']] = np.abs(row['EdgeWeight'])
leap_matrix = pd.DataFrame(0.0, index=common_gene, columns=common_tf)
for index, row in leap_df.iterrows():
    leap_matrix.at[row['Gene2'], row['Gene1']] = np.abs(row['EdgeWeight'])
pidc_matrix = pd.DataFrame(0.0, index=common_gene, columns=common_tf)
for index, row in pidc_df.iterrows():
    pidc_matrix.at[row['Gene2'], row['Gene1']] = np.abs(row['EdgeWeight'])
ppcor_matrix = pd.DataFrame(0.0, index=common_gene, columns=common_tf)
for index, row in ppcor_df.iterrows():
    ppcor_matrix.at[row['Gene2'], row['Gene1']] = np.abs(row['EdgeWeight'])
scode_matrix = pd.DataFrame(0.0, index=common_gene, columns=common_tf)
for index, row in scode_df.iterrows():
    scode_matrix.at[row['Gene2'], row['Gene1']] = np.abs(row['EdgeWeight'])
scribe_matrix = pd.DataFrame(0.0, index=common_gene, columns=common_tf)
for index, row in scribe_df.iterrows():
    scribe_matrix.at[row['Gene2'], row['Gene1']] = np.abs(row['EdgeWeight'])
sincerities_matrix = pd.DataFrame(0.0, index=common_gene, columns=common_tf)
for index, row in sincerities_df.iterrows():
    sincerities_matrix.at[row['Gene2'], row['Gene1']] = np.abs(row['EdgeWeight'])
# singe_matrix = pd.DataFrame(0.0, index=common_gene, columns=common_tf)
# for index, row in singe_df.iterrows():
#     singe_matrix.at[row['Gene2'], row['Gene1']] = np.abs(row['EdgeWeight'])
op_matrix = op_df[common_tf]
op_matrix = op_matrix.loc[common_gene]
inf_matrix = inf_df[common_tf]
inf_matrix = inf_matrix.loc[common_gene]

In [ ]:
from scipy import stats
steady_state_df = pd.read_csv(set_dir + 'dualknockouts.tsv', sep='\t')

In [ ]:
for regulator_name in tqdm(common_tf):
    for target_name in common_gene:
        edge_name = regulator_name + '_' + target_name
        ss_gene = steady_state_df[target_name]
        ss_tf = steady_state_df[regulator_name]
        corr, _ = stats.pearsonr(ss_tf, ss_gene)
        ss_ranked, _ =  stats.spearmanr(ss_tf, ss_gene)
        ss_cor = corr**2
        op_score = op_matrix.loc[target_name][regulator_name]
        inf_score = inf_matrix.loc[target_name][regulator_name]
        leap_score = leap_matrix.loc[target_name][regulator_name]
        pidc_score = pidc_matrix.loc[target_name][regulator_name]
        boost_score = boost_matrix.loc[target_name][regulator_name]
        genie3_score = genie3_matrix.loc[target_name][regulator_name]
        sincerities_score = sincerities_matrix.loc[target_name][regulator_name]
        ppcor_score = ppcor_matrix.loc[target_name][regulator_name]
        grisli_score = grisli_matrix.loc[target_name][regulator_name]
        grnvbem_score = grnvbem_matrix.loc[target_name][regulator_name]
        scode_score = scode_matrix.loc[target_name][regulator_name]
        scribe_score = scribe_matrix.loc[target_name][regulator_name]
        # singe_score = singe_matrix.loc[target_name][regulator_name]
        target = 0
        if edge_name in target_edge_set:
            target = 1
        # ensemble_df = ensemble_df.append(pd.DataFrame([[edge_name, op_score, inf_score, boost_score, genie3_score, grisli_score, grnvbem_score, leap_score, pidc_score, ppcor_score, scode_score, scribe_score, sincerities_score, singe_score, ss_cor, ss_ranked, target]], columns=['edge_name', 'op', 'inf', 'grnboost', 'genie3', 'grisli', 'grnvbem', 'leap',  'pidc', 'ppcor', 'scode', 'scribe', 'sincerities', 'singe', 'ss_cor', 'ss_ranked', 'edge_exist']), ignore_index=True)
        ensemble_df = ensemble_df.append(pd.DataFrame([[edge_name, op_score, inf_score, boost_score, genie3_score, grisli_score, grnvbem_score, leap_score, pidc_score, ppcor_score, scode_score, scribe_score, sincerities_score, ss_cor, ss_ranked, target]], columns=['edge_name', 'op', 'inf', 'grnboost', 'genie3', 'grisli', 'grnvbem', 'leap',  'pidc', 'ppcor', 'scode', 'scribe', 'sincerities', 'ss_cor', 'ss_ranked', 'edge_exist']), ignore_index=True)
        # ensemble_df = ensemble_df.append(pd.DataFrame([[edge_name, op_score, inf_score, boost_score, genie3_score, grisli_score, grnvbem_score, leap_score, pidc_score, ppcor_score, scode_score, scribe_score, ss_cor, ss_ranked, target]], columns=['edge_name', 'op', 'inf', 'grnboost', 'genie3', 'grisli', 'grnvbem', 'leap',  'pidc', 'ppcor', 'scode', 'scribe', 'ss_cor', 'ss_ranked', 'edge_exist']), ignore_index=True)

100%|██████████| 50/50 [00:05<00:00,  9.90it/s]


In [ ]:
ensemble_df = ensemble_df.astype({'edge_exist': 'int8'})
ensemble_df.to_csv(set_dir + 'ensemble.csv', index=False)